In [23]:
import pandas as pd
dataset=pd.read_csv('../tratamento/dataset_tratado.csv')

dataset_ibge=pd.read_csv('../tratamento/dataset_ibge_tratado.csv')

In [24]:
#Removendo caracteres indesejados no Nome 
dataset['Nome'] = dataset['Nome'].replace({',': ''}, regex=True)
dataset['Nome'] = dataset['Nome'].replace({'-': ''}, regex=True)
dataset = dataset.replace({'  ': ' '}, regex=True)
dataset['Nome'] = dataset['Nome'].replace({"'":' '}, regex=True)



In [25]:
#Valida os CPFS

from validate_docbr import CPF
#Valida os CPFS 
cpf = CPF()
def validaCPF(CPF):
    
    if(cpf.validate(CPF)):
        CPFValido ="Sim"
    else:
        CPFValido ="Não"

        
    return CPFValido

dataset['Validade CPF']=dataset['cpf'].apply(validaCPF)


In [26]:
#Seleciona apenas os CPFs validos
dataset=dataset.loc[dataset['Validade CPF']=="Sim"]
dataset.drop(columns=['Validade CPF'],inplace=True)


In [27]:
#Utiliza Regex para pegar o utimo conjunto de caracteres do nome depois do espaço
import re

def retornaSobrenome(nome): 

  Sobrenome = re.search("[^\/]\w+$", nome).group(0)
  #print(Sobrenome)
  return Sobrenome

dataset['sobrenome']=dataset['Nome'].apply(retornaSobrenome)

In [28]:
#fazendo um join com a base do IBGE para adionar os campos Cod_IBGE,sigla , Estado 
dataset=pd.merge(dataset,dataset_ibge,how='left',on='Cidade')

In [29]:
#Renomeando as colunas 
dataset.rename(columns={'CPF':'CPF','sobrenome':'Sobrenome','cod_ibge':'Cod_IBGE',
'sigla':'Sigla','Nome Sigla':'Estado'},inplace=True)


In [30]:
Dim_localidade=dataset.loc[:,['Cod_IBGE','Sigla','Estado']]
dataset.drop(columns=['Sigla','Estado'],inplace=True)




In [31]:
Fact_indicados=dataset.loc[dataset['Tipo Cargo']=="Indicado"]

In [32]:
df01=dataset.loc[dataset['Tipo Cargo']=="Eleito"]
df02=dataset.loc[dataset['Tipo Cargo']=="Concursado"]

Dim_indicadores=pd.concat([df01,df02])


In [33]:
df01=Fact_indicados.merge(Dim_indicadores,how='inner',on='Sobrenome')

In [ ]:
Dim_indicadores.to_csv(r'../processamento/Dim_indicadoresv.csv', index=False)
Fact_indicados.to_csv(r'../processamento/Fact_indicados.csv', index=False)
Dim_localidade.to_csv(r'../processamento/Dim_localidade.csv', index=False)